In [102]:
import numpy as np

In [135]:
global inp,ker,in_r,in_col,in_chn,out_r,out_col
in_r = 64
in_col  = 64
in_chn = 134
out_r = in_r-2
out_col = in_col-2

inp = np.random.uniform(low = -10,
                           high = 10,
                           size = (in_chn,in_r,in_col) #size = (chn,rows,cols)
                           ).astype(np.float32).tolist()


ker = np.random.uniform(low = -4,
                           high = 4,
                           size = (in_chn,3,3) #size = (chn,rows,cols)
                           ).astype(np.float32).tolist()


# inp = np.ones((32,14,14),dtype=float).tolist()


# ker = np.ones((32,3,3),dtype=float).tolist()

In [104]:
convOut = np.zeros((out_r,out_col),dtype=float).tolist()
convOutPool = np.zeros((out_r-2,out_r-2),dtype=float).tolist()

In [119]:
def fetchGroup(chn,row,col,inter_pipe):
  for j in range(col,col+3):
    for i in range(row,row+3):
      for k in range(chn,chn+8):
        if(k>=in_chn):
          inter_pipe.append(0)
          continue
        ##for padding, whenever we are on 0th or last row/col, we return a 0 cylinder
        if(i==0 or j==0 or i == (in_r+1) or j==(in_col+1)):
          inter_pipe.append(0)
        else:
          inter_pipe.append(inp[k][i-1][j-1])
  return

In [107]:
def convolveGroup(inGroup,kerGroup):
  temp_out = 0
  for i in range(len(inGroup)):
    temp_out+=(inGroup[i] * kerGroup[i])
  return temp_out

In [131]:
def fetchKernel(chn,kern_pipe):
  for j in range(3):
    for i in range(3):
      for k in range(chn,chn+8):
        if(k>=in_chn):
          kern_pipe.append(0)
          continue
        kern_pipe.append(ker[k][i][j])
  return

In [109]:
def applyActivation(out):
  if out>0:
    return out
  return 0

In [123]:
def sendPool(pool_buffer_r0,pool_buffer_r1, pool_buffer_r2,final_pool_buffer):
  assert len(final_pool_buffer) == 0
  inter_maxPool_buffer = []
  pool_buf_len = len(pool_buffer_r0)

  for i in range(pool_buf_len):
    r0_val = pool_buffer_r0.pop(0)
    r1_val = pool_buffer_r1.pop(0)
    r2_val = pool_buffer_r2.pop(0)
    inter_maxPool_buffer.append(max(r0_val,r1_val,r2_val))
    pool_buffer_r0.append(r1_val)
    pool_buffer_r1.append(r2_val)
  # print(inter_maxPool_buffer)
  assert len(pool_buffer_r0) == pool_buf_len
  assert len(pool_buffer_r2) == 0

  max_val_col0 = inter_maxPool_buffer.pop(0)
  max_val_col1 = inter_maxPool_buffer.pop(0)
  max_val_col2 = inter_maxPool_buffer.pop(0)

  final_pool_buffer.append(max(max_val_col0,max_val_col1,max_val_col2))

  max_val_col0 = max_val_col1
  max_val_col1 = max_val_col2

  inter_buf_len = len(inter_maxPool_buffer)
  for i in range(inter_buf_len):
    max_val_col2 = inter_maxPool_buffer.pop(0)
    final_pool_buffer.append(max(max_val_col0,max_val_col1,max_val_col2))
    max_val_col0 = max_val_col1
    max_val_col1 = max_val_col2

  assert len(inter_maxPool_buffer) == 0
  assert len(final_pool_buffer) == pool_buf_len-2
  return

In [136]:
import time
fetch_time = []
comp_time = []

depth = in_chn//8
if(in_chn%8 != 0 ):
  depth+=1


# Parameters for pooling
pool = True
pool_buffer_r0 = []
pool_buffer_r1 = []
pool_buffer_r2 = []
cur_pool_row = 0
final_pool_buffer = []

# Parameters for Activation
activation = False

# Parameters for padding
pad = True
if pad:
  out_r = in_r
  out_col = in_col
  convOut = np.zeros((out_r,out_col),dtype=float).tolist()
  convOutPool = np.zeros((out_r-2,out_r-2),dtype=float).tolist()

for r in range(out_r):
  for c in range(out_col):
    out = 0

    for chn in range(depth):
      inter_pipe = []
      in_pipe = []
      ker_pipe = []
      # Fetch Input Group
      start = time.time()
      fetchGroup(chn*8,r,c,inter_pipe)
      end = time.time()
      fetch_time.append(end - start)

      pipeLen = len(inter_pipe)
      for i in range(pipeLen):
        in_pipe.append(inter_pipe.pop(0))

      # Fetch Kernel
      fetchKernel(chn*8,ker_pipe)

      start = time.time()
      out += convolveGroup(in_pipe,ker_pipe)
      end = time.time()
      comp_time.append(end - start)
    # Apply non-activation if required
    if activation:
      out = applyActivation(out)
    # Apply Pooling if required
    if pool:
      if r == 0:
        pool_buffer_r0.append(out)
      elif r == 1:
        pool_buffer_r1.append(out)
      else:
        pool_buffer_r2.append(out)
        if c >= (out_col-1):
          sendPool(pool_buffer_r0,pool_buffer_r1, pool_buffer_r2,final_pool_buffer)
          pool_buf_cols =  len(final_pool_buffer)
          for cur_pool_col in range(pool_buf_cols):
            convOutPool[cur_pool_row][cur_pool_col] = final_pool_buffer.pop(0)
          cur_pool_row+=1

    convOut[r][c] = out

print(convOut[0])
if pool:
  print(convOutPool[0])


[-363.1174184196851, 104.94978954107145, -355.1291418964718, -924.0769430759444, -155.4464010109909, -333.42117131944354, 501.18174197954806, 133.18448941015598, -268.3455394920324, 365.0226975411879, 599.6275772173351, -281.7766485083101, -650.9587847152724, 103.21439275398613, 160.37592718418773, -474.73471713902074, 80.40148359626548, -420.5356775990176, 343.16970692892716, 496.80809355376675, -426.4424001846249, 141.94717869374352, 145.9058824075056, 144.6381067372473, -22.439364306256223, 301.7159581359158, 408.77506290354177, -18.78915611720123, 298.05081342772223, -611.1996227254696, 14.61247191602638, 473.2489849078562, 236.43997471765812, 196.33937312457073, -179.29107080629942, 189.69068928799746, -197.84512933762602, 60.53507617302495, 306.2128686695562, 469.15000971315925, -24.2751546328721, -152.0671123979198, 149.88744197636248, 515.8300689992238, -75.13934231374733, 195.05276702387943, 276.9291945730574, -478.4243370368022, 263.8283850420777, 324.70880966399096, 96.46882

In [137]:
# print(fetch_time)
# print(comp_time)
print("Total Fetch Time ",sum(fetch_time)*1000," ms")
print("Total Compute Time ",sum(comp_time)*1000," ms")

Total Fetch Time  2021.1544036865234  ms
Total Compute Time  589.8172855377197  ms


In [138]:
import torch
import torch.nn.functional as F

new_inp = np.expand_dims(np.array(inp),axis=0)
new_ker = np.expand_dims(np.array(ker),axis=0)

inp_tensor = torch.from_numpy(new_inp)
ker_tensor = torch.from_numpy(new_ker)
# tensor_out = F.conv2d(input=inp_tensor,weight=ker_tensor)
tensor_out = F.conv2d(input=inp_tensor,weight=ker_tensor,padding = 1)
tensor_pool_out = F.max_pool2d(input = tensor_out,kernel_size=3,stride=1)
torch_imp_tensor = tensor_out[0][0]
print(torch_imp_tensor[0])
if pool:
  torch_imp_pool_tensor = tensor_pool_out[0][0]
  print(torch_imp_pool_tensor[0])

tensor([-363.1174,  104.9498, -355.1291, -924.0769, -155.4464, -333.4212,
         501.1817,  133.1845, -268.3455,  365.0227,  599.6276, -281.7766,
        -650.9588,  103.2144,  160.3759, -474.7347,   80.4015, -420.5357,
         343.1697,  496.8081, -426.4424,  141.9472,  145.9059,  144.6381,
         -22.4394,  301.7160,  408.7751,  -18.7892,  298.0508, -611.1996,
          14.6125,  473.2490,  236.4400,  196.3394, -179.2911,  189.6907,
        -197.8451,   60.5351,  306.2129,  469.1500,  -24.2752, -152.0671,
         149.8874,  515.8301,  -75.1393,  195.0528,  276.9292, -478.4243,
         263.8284,  324.7088,   96.4688,  -24.2699, -152.4903,  231.5113,
        -297.9125,  475.8790, -193.2499, -279.0776,   65.7538,   23.1479,
         -49.5412, -320.0308,  278.5582,  124.4809], dtype=torch.float64)
tensor([ 549.9942,  549.9942,  644.3797,  644.3797,  644.3797,  501.1817,
        1075.0637, 1075.0637, 1075.0637,  677.7543,  677.7543,  103.2144,
         746.0011,  746.0011,  746.001

In [139]:
our_impl_tensor = torch.from_numpy(np.array(convOut))
diff = torch.sub(our_impl_tensor,torch_imp_tensor)
# our_impl_tensor = torch.from_numpy(np.array(convOutPool))
# diff = torch.sub(our_impl_tensor,torch_imp_pool_tensor)
print("Max difference between our implementation and torch implementation ",torch.max(diff))
print("Min difference between our implementation and torch implementation ",torch.min(diff))

Max difference between our implementation and torch implementation  tensor(2.5011e-12, dtype=torch.float64)
Min difference between our implementation and torch implementation  tensor(-3.1832e-12, dtype=torch.float64)


In [ ]:

diff_indices = torch.nonzero(diff)
for indi in diff_indices:
  row = indi[0]
  col = indi[1]
  print("Value I got ",our_impl_tensor[row][col])
  print("Value pytorch got ",torch_imp_tensor[row][col])
  print("Difference Value is ",diff[row][col])
  meas = our_impl_tensor[row][col]
  real = torch_imp_tensor[row][col].numpy()
  error = abs(diff[row][col].numpy())
  print("Relative error is ", abs(error/real)*100)
  print()